In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [5]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [10]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


1: Calcular Team Strenght

In [12]:
#Dividimos df en df_home y df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away=df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]
df_home

,HomeTeam,HomeGoals,AwayGoals
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
895,Brazil,2,0
896,Serbia,1,2
897,Serbia,0,2
898,France,1,0


In [13]:
#Renombramos las columnas:
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScore','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals ':'GoalsConceded'})


In [19]:
#Concatenamos df_home y df_away, hacemos group por team y calculamos el promedio.
df_team_strength=pd.concat([df_home,df_away], ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScore,GoalsConceded,AwayGoals
Team,,,
Algeria,0.833333,1.461538,1.142857
Angola,0.000000,0.666667,0.500000
Argentina,1.913793,1.148148,1.130435
Australia,1.000000,1.937500,0.666667
Austria,2.285714,1.620690,0.733333
...,...,...,...
Uruguay,2.060606,1.321429,0.826087
Wales,2.000000,0.800000,0.500000
West Germany,2.500000,1.241935,1.500000


2: Funcion predict_points:

# Distribucion de Poisson: 
Es una distribucion discreta que describe el numero de eventos que ocurren en un intervalo de tiempo fijo o region de oportunidad.

Ejemplo: Goal, evento que puede ocurrin en los 90 minutos de un partido de futbol.

Condiciones:
1. El numero de eventos se puede contar
2. La ocurrencia de eventos son independientes
3. La tasa a la que ocurren los eventos es constante
4. Dos eventos no pueden ocurrir exactamente en el mismo instante de tiempo

In [22]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScore'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScore'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

Testeamos la funcion:

In [23]:
predict_points('Argentina','Mexico')

(2.3492544168244787, 0.5148996815982367)

3: Prediciendo Mundial

Fase de grupos:

In [24]:
#Dividiendo fixture en grupo, octavos, cuatros, ...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [25]:
#Correr todos los partidos de la fase de grupo y actualizar las tablas de cada grupo.
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [26]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Ecuador,2.0
2,Senegal,2.0
3,Qatar (H),0.0


Octavos:

In [27]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [28]:
#Actualizamos el fixture de octavos con el 1 puesto y el 2 puesto.
for group in dict_table:
    group_winner = dict_table[group].loc[0,'Team']
    Runners_up = dict_table[group].loc[1,'Team']
    df_fixture_knockout.replace({f'Winners {group}': group_winner,
                                 f'Runners-up {group}': Runners_up}, inplace=True)
    
df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,England,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,Wales,Match 51,Ecuador,2022,?
52,Germany,Match 53,Croatia,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Belgium,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [29]:
#Cramos funciones get_winner:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [30]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,England,2022,England
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,Wales,Match 51,Ecuador,2022,Wales
52,Germany,Match 53,Croatia,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Belgium,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


Quartos de final:

In [31]:
#Creamos la funcion de update_table
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [32]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,England,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,Wales,Match 59,France,2022,?


In [33]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,England,Match 57,Argentina,2022,England
58,Spain,Match 60,Portugal,2022,Portugal
59,Wales,Match 59,France,2022,Wales


Semifinal:

In [34]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,England,Match 61,Brazil,2022,?
61,Wales,Match 62,Portugal,2022,?


In [35]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,England,Match 61,Brazil,2022,Brazil
61,Wales,Match 62,Portugal,2022,Wales


## FINAL:

In [36]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,Wales,2022,?


In [37]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,Wales,2022,Wales
